# Apriori Algorithm Implementation Assignment

### Objective:
You will implement the **Apriori algorithm** from scratch (i.e., without using any libraries like `mlxtend`) to find frequent itemsets and generate association rules.

### Dataset:
Use the [Online Retail Dataset](https://www.kaggle.com/datasets/vijayuv/onlineretail) from Kaggle. You can filter it for a specific country (e.g., `United Kingdom`) and time range to reduce size if needed.

---

## Step 1: Data Preprocessing

- Load the dataset
- Remove rows with missing values
- Filter out rows where `Quantity <= 0`
- Convert Data into Basket Format

👉 **Implement code below**

In [1]:
import pandas as pd

In [9]:
# Load the dataset
df = pd.read_csv('OnlineRetail.csv' ,encoding='ISO-8859-1')
# Preprocess as per the instructions above | We have already done in TASK 2

# Your Code Here
# for Basket

basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
# basket = pd.read_csv("./basket.csv")

basket = basket.applymap(lambda x: 1 if x > 0 else 0)
basket

C:\Users\ASUS\AppData\Local\Temp\ipykernel_13112\589385635.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C581484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 2: Implement Apriori Algorithm
Step-by-Step Procedure:
1. Generate Frequent 1-Itemsets
Count the frequency (support) of each individual item in the dataset.
Keep only those with support ≥ min_support.
→ Result is L1 (frequent 1-itemsets)
2. Iterative Candidate Generation (k = 2 to n)
While L(k-1) is not empty:
a. Candidate Generation

Generate candidate itemsets Ck of size k from L(k-1) using the Apriori property:
Any (k-itemset) is only frequent if all of its (k−1)-subsets are frequent.
b. Prune Candidates
Eliminate candidates that have any (k−1)-subset not in L(k-1).
c. Count Support
For each transaction, count how many times each candidate in Ck appears.
d. Generate Frequent Itemsets
Form Lk by keeping candidates from Ck that meet the min_support.
Repeat until Lk becomes empty.
Implement the following functions:
1. `get_frequent_itemsets(transactions, min_support)` - Returns frequent itemsets and their support
2. `generate_candidates(prev_frequent_itemsets, k)` - Generates candidate itemsets of length `k`
3. `calculate_support(transactions, candidates)` - Calculates the support count for each candidate

**Write reusable functions** for each part of the algorithm.

In [10]:
# Implement apriori functions below

def get_frequent_itemsets(transactions, min_support):
    """
    Generates frequent itemsets using the Apriori algorithm.

    Args:
        transactions (pd.DataFrame): DataFrame with transactions as rows and items as columns (binary).
        min_support (float): Minimum support threshold.

    Returns:
        dict: A dictionary where keys are itemset lengths (k) and values are dictionaries
              of frequent itemsets (as frozensets) and their support counts.
    """
    frequent_itemsets = {}
    k = 1

    # Generate frequent 1-itemsets
    item_counts = transactions.sum(axis=0)
    L1 = {frozenset({item}): count for item, count in item_counts.items() if count / len(transactions) >= min_support}
    frequent_itemsets[k] = L1

    # Iteratively generate frequent k-itemsets
    while frequent_itemsets[k]:
        k += 1
        Ck = generate_candidates(frequent_itemsets[k-1], k)
        support_Ck = calculate_support(transactions, Ck)
        Lk = {itemset: support for itemset, support in support_Ck.items() if support / len(transactions) >= min_support}
        frequent_itemsets[k] = Lk

    return frequent_itemsets

def generate_candidates(prev_frequent_itemsets, k):
    """
    Generates candidate itemsets of length k from frequent (k-1)-itemsets.

    Args:
        prev_frequent_itemsets (dict): Dictionary of frequent (k-1)-itemsets and their support.
        k (int): The desired length of candidate itemsets.

    Returns:
        set: A set of candidate itemsets (as frozensets).
    """
    candidates = set()
    for itemset1 in prev_frequent_itemsets.keys():
        for itemset2 in prev_frequent_itemsets.keys():
            union = itemset1.union(itemset2)
            if len(union) == k:
                # Check if all (k-1)-subsets are in the previous frequent itemsets
                is_valid_candidate = True
                for item in union:
                    subset = union - {item}
                    if subset not in prev_frequent_itemsets:
                        is_valid_candidate = False
                        break
                if is_valid_candidate:
                    candidates.add(union)
    return candidates


def calculate_support(transactions, candidates):
    """
    Calculates the support count for each candidate itemset.

    Args:
        transactions (pd.DataFrame): DataFrame with transactions as rows and items as columns (binary).
        candidates (set): A set of candidate itemsets (as frozensets).

    Returns:
        dict: A dictionary of candidate itemsets and their support counts.
    """
    support_counts = {}
    for candidate in candidates:
        # Check if the candidate itemset is present in each transaction
        is_subset = transactions[list(candidate)].all(axis=1)
        support_counts[candidate] = is_subset.sum()
    return support_counts

## Step 3: Generate Association Rules

- Use frequent itemsets to generate association rules
- For each rule `A => B`, calculate:
  - **Support**
  - **Confidence**
- Only return rules that meet a minimum confidence threshold (e.g., 0.5)

👉 **Implement rule generation function below**

In [ ]:
# Function to generate rules from frequent itemsets

def generate_rules(frequent_itemsets, min_confidence):
    """
    Generates association rules from frequent itemsets.

    Args:
        frequent_itemsets (dict): A dictionary where keys are itemset lengths (k) and values are dictionaries
                                  of frequent itemsets (as frozensets) and their support counts.
        min_confidence (float): Minimum confidence threshold.

    Returns:
        list: A list of dictionaries, where each dictionary represents a rule with 'antecedent',
              'consequent', 'support', and 'confidence'.
    """
    rules = []
    # Iterate through frequent itemsets of size 2 or more
    for k in sorted(frequent_itemsets.keys()):
        if k > 1:
            for itemset, support in frequent_itemsets[k].items():
                for consequent in itemset:
                    antecedent = itemset - {consequent}
                    if antecedent in frequent_itemsets[k-1]:
                        antecedent_support = frequent_itemsets[k-1][antecedent]
                        confidence = support / antecedent_support
                        if confidence >= min_confidence:
                            rules.append({
                                'antecedent': set(antecedent),
                                'consequent': set({consequent}),
                                'support': support / len(basket),  # Calculate support as a fraction
                                'confidence': confidence
                            })
    return rules

## Step 4: Output and Visualize

- Print top 10 frequent itemsets
- Print top 10 association rules (by confidence or lift)

👉 **Output results below**

In [ ]:
# Output the final results
# Optional: Add visualizations

# Your Code Here

# Set minimum support and confidence
min_support = 0.01  # You can adjust this threshold
min_confidence = 0.5 # You can adjust this threshold

# Get frequent itemsets
frequent_itemsets = get_frequent_itemsets(basket, min_support)

# Generate association rules
rules = generate_rules(frequent_itemsets, min_confidence)

# Print top 10 frequent itemsets (by support)
print("Top 10 Frequent Itemsets:")
all_frequent_itemsets = []
for k, itemsets in frequent_itemsets.items():
    for itemset, support in itemsets.items():
        all_frequent_itemsets.append({'itemset': itemset, 'support': support / len(basket)})

sorted_itemsets = sorted(all_frequent_itemsets, key=lambda x: x['support'], reverse=True)
for itemset_info in sorted_itemsets[:10]:
    print(f"Itemset: {set(itemset_info['itemset'])}, Support: {itemset_info['support']:.4f}")

print("\nTop 10 Association Rules (by Confidence):")
# Sort rules by confidence
sorted_rules_confidence = sorted(rules, key=lambda x: x['confidence'], reverse=True)
for rule in sorted_rules_confidence[:10]:
    print(f"Rule: {rule['antecedent']} => {rule['consequent']}, Confidence: {rule['confidence']:.4f}, Support: {rule['support']:.4f}")

print("\nTop 10 Association Rules (by Support):")
# Sort rules by support
sorted_rules_support = sorted(rules, key=lambda x: x['support'], reverse=True)
for rule in sorted_rules_support[:10]:
    print(f"Rule: {rule['antecedent']} => {rule['consequent']}, Confidence: {rule['confidence']:.4f}, Support: {rule['support']:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize top 10 frequent itemsets by support
top_itemsets_df = pd.DataFrame(sorted_itemsets[:10])
top_itemsets_df['itemset_str'] = top_itemsets_df['itemset'].apply(lambda x: ', '.join(list(x)))

plt.figure(figsize=(12, 6))
sns.barplot(x='support', y='itemset_str', data=top_itemsets_df, palette='viridis')
plt.title('Top 10 Frequent Itemsets by Support')
plt.xlabel('Support')
plt.ylabel('Itemset')
plt.show()

In [ ]:
# Visualize top 10 association rules by confidence
top_rules_confidence_df = pd.DataFrame(sorted_rules_confidence[:10])
top_rules_confidence_df['rule_str'] = top_rules_confidence_df.apply(lambda row: f"{list(row['antecedent'])} => {list(row['consequent'])}", axis=1)

plt.figure(figsize=(12, 6))
sns.barplot(x='confidence', y='rule_str', data=top_rules_confidence_df, palette='viridis')
plt.title('Top 10 Association Rules by Confidence')
plt.xlabel('Confidence')
plt.ylabel('Rule')
plt.show()

In [ ]:
# Visualize top 10 association rules by support
top_rules_support_df = pd.DataFrame(sorted_rules_support[:10])
top_rules_support_df['rule_str'] = top_rules_support_df.apply(lambda row: f"{list(row['antecedent'])} => {list(row['consequent'])}", axis=1)

plt.figure(figsize=(12, 6))
sns.barplot(x='support', y='rule_str', data=top_rules_support_df, palette='viridis')
plt.title('Top 10 Association Rules by Support')
plt.xlabel('Support')
plt.ylabel('Rule')
plt.show()